![Fluxo de trabalho para geração aumentada de recuperação RAG](img/download.jfif')

In [1]:
import os
import glob
from typing import List
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import PyPDFLoader


from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def create_chat(t=0.0):
    """
    Configura os objetos do AzureChatOpenAI para serem utilizados.
 
    Args:
        t (float): O valor da temperatura para determinar o comportamento da resposta, por padrão é 0.0.
   
    Returns:
        AzureChatOpenAI: O objeto de AzureChatOpenAi configurado.
    """
    llm_chat = AzureChatOpenAI(openai_api_base=os.getenv("OPENAI_API_BASE"),
                    openai_api_version=os.getenv("OPENAI_API_VERSION"),
                    openai_api_key=os.getenv("OPENAI_API_KEY"),
                    openai_api_type=os.getenv("OPENAI_API_TYPE"),
                    deployment_name=os.getenv("DEPLOYMENT_NAME"),
                    temperature=t
    )
    return llm_chat

In [3]:
# load pdf
loaders = [
    PyPDFLoader(r'C:\AI\Cursos\LangChain_Chat_Data\data\Docker_para_desenvolvedores.pdf'),
           PyPDFLoader(r'C:\AI\Cursos\LangChain_Chat_Data\data\Containers_com_Docker.pdf')
        ]   
           
docs = []

for loader in loaders:
    docs.extend(loader.load())

In [4]:
# split

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
)

splits = text_splitter.split_documents(docs)
# Total de chunks
len(splits)

336

In [5]:
# Embeddings - Vamos incorporar nossas divisões

def create_embeddings():
    embeddings = OpenAIEmbeddings(
        deployment = os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
    )
    return embeddings

embedding = create_embeddings()

## Exemplo de criação de Embeddings

In [6]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [7]:
# Criar a representação numérica daquele texto
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [8]:
import numpy as np

In [9]:
# Os dois primeiros textos são bem parecidos
np.dot(embedding1, embedding2)

0.9632123594065702

In [10]:
# Não tão parecidos
np.dot(embedding1, embedding3)

0.7710794706526474

In [11]:
# Não tão parecidos
np.dot(embedding2, embedding3)

0.7597185772197995

In [12]:
from langchain.vectorstores import Chroma

In [13]:
#persist_directory = r'data\chroma'

In [14]:
! rm -rf ./docs/chroma

'rm' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [15]:
vectordb = Chroma.from_documents(
    
    documents=splits,
    embedding=embedding,
    #persist_directory=persist_directory
)

In [16]:
# Mesma quantidade de splits
print(vectordb._collection.count())

336


In [17]:
query = "Do que se trata os arquivos?"

# k=3 especifica o número de documents que serão retornados
docs = vectordb.similarity_search(query,k=3)

In [18]:
len(docs)

3

In [19]:
# Busca por similaridade do primeiro documento
print(docs[0].page_content)

Capítulo 4 
Trabalhando com volumes 
 
 
Um volume pode ser um diretório localizado fora do sistema de arquivos de 
um container . O Docker permite especificar diretórios no container  para que 
possam ser mapeados no sistema de arquivos do host. Com isso, temos a 
capacidade de manipular dados no container  sem que tenham relação alguma 
com as informações da imagem. Um volume pode ser compartilhado entre 
vários containers . A figura a seguir ilustra como os volumes funcionam:


In [20]:
#vectordb.persist()

In [21]:
print(docs[1].metadata)

{'page': 50, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Docker_para_desenvolvedores.pdf'}


In [22]:
query = "Qual a diferença entre Containers, Docker e imagens?"

docs = vectordb.similarity_search(query,k=5)

for doc in docs:
    print(doc.metadata)

{'page': 40, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Docker_para_desenvolvedores.pdf'}
{'page': 14, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Docker_para_desenvolvedores.pdf'}
{'page': 18, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Docker_para_desenvolvedores.pdf'}
{'page': 160, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Docker_para_desenvolvedores.pdf'}
{'page': 15, 'source': 'C:\\AI\\Cursos\\LangChain_Chat_Data\\data\\Containers_com_Docker.pdf'}


In [25]:
print(docs[4].page_content)

1.2. O que diferencia um container de uma máquina virtual? Casa do Código 
 
1.2 O que diferencia um container de uma máquina virtual? 
Para entender melhor a diferença entre virtualização e containers , vamos 
relembrar primeiro os tipos de virtualização. 
Em resumo, temos os seguintes tipos: 
• Bare Metal ; 
• Hosted . 
 
No Bare Metal , o software  que proporciona a virtualização é instalado 
diretamente sobre o hardware, por exemplo, Xen, VMware ESXi e Hyper-V . 
Esse tipo de virtualização proporciona um isolamento maior e, ao mesmo 
tempo, uma sobrecarga, pois cada máquina virtual que é criada executará seu próprio kernel e instância do sistema operacional. Já o tipo hosted , o 
software  que proporciona a virtualização é executado sobre um sistema 
operacional, por exemplo, o VirtualBox . 
A virtualização por containers , proposta pelo LXC, ocorre de forma 
menos isolada, pois compartilha algumas partes do kernel do host, fazendo 
com que a sobrecarga seja menor. 
Na figura a seg